In [ ]:
import numpy as np 
import pandas as pd
import os
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import Input
from sklearn.utils import class_weight
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Activation
from tensorflow.keras.layers import BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import shuffle

In [ ]:
df_train = pd.read_csv("../input/csv-knowledge-distillation/image-labels_train.csv")
df_train = shuffle(df_train, random_state=41)
df_train["label"] = df_train["label"].astype("str")
df_test = pd.read_csv("../input/csv-knowledge-distillation/image-labels_test.csv")
df_test = shuffle(df_test, random_state=31)
df_test["label"] = df_test["label"].astype("str")
df_train.head(4)
df_test.head(4)

In [ ]:
#Resizing image for EfficientNetB5 and image augmentation
target_size = (200, 200) #EfficientNetB5 supports 456*456 pixel images 

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.2, 
    width_shift_range=0.1,
    height_shift_range=0.1
) 

#Train_Data 
train_generator = train_datagen.flow_from_dataframe(
    dataframe = df_train,
    x_col='image_id',
    y_col='label',
    directory="../input/merged-chest-xray-data/chest_xray/train",
    target_size=target_size,
    color_mode="grayscale",
    batch_size=64,
    shuffle=True,
    class_mode='categorical') 

#Test_Data

validation_datagen = ImageDataGenerator(rescale=1. / 255)
validation_generator = validation_datagen.flow_from_dataframe(
    dataframe = df_test,
    x_col='image_id',
    y_col='label',
    directory='../input/merged-chest-xray-data/chest_xray/test',
    target_size=target_size,
    color_mode="grayscale",
    shuffle=False,
    batch_size=64,
    class_mode='categorical')

In [ ]:
""""from tensorflow.keras.models import Sequential
import tensorflow as tf
from tensorflow.keras import layers

img_augmentation = Sequential(
    [   
        tf.keras.layers.experimental.preprocessing.RandomRotation(factor=0.15),
        tf.keras.layers.experimental.preprocessing.RandomRotation(factor=0.15),
        tf.keras.layers.experimental.preprocessing.RandomTranslation(height_factor=0.1, width_factor=0.1),
        tf.keras.layers.experimental.preprocessing.RandomFlip(),
        tf.keras.layers.experimental.preprocessing.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
) 
"""


In [ ]:
""""
IMG_SIZE = 456 
from tensorflow.keras.applications import EfficientNetB5 

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = img_augmentation(inputs)
    outputs = EfficientNetB5(include_top=True, weights=None, classes=2)(x)

    model = tf.keras.Model(inputs, outputs)
    model.compile(
        optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
    )

model.summary()
epochs = 4  
hist = model.fit(train_loader, epochs=epochs, validation_data=test_loader, verbose=2) 
"""

In [ ]:
classes_to_predict = sorted(df_train.label.unique())

In [ ]:
from tensorflow.keras.applications import EfficientNetB5

effnetb5 = EfficientNetB5(weights=None,include_top=False,input_shape=(200, 200, 1))
#effnet.load_weights("../input/efficientnet-keras-weights-b0b5/efficientnet-b5_imagenet_1000_notop.h5")
for layer in effnetb5.layers:
    layer.trainable = True

In [ ]:
model = Sequential()
model.add(effnetb5) 
model.add(GlobalAveragePooling2D())
model.add(Dense(128))
model.add(Activation('relu'))

model.add(Dense(len(classes_to_predict), activation="softmax"))
    
model.summary()

In [ ]:
callbacks = [ReduceLROnPlateau(monitor='loss_value', patience=1, verbose=1, factor=0.5),
             EarlyStopping(monitor='loss_value', patience=2),
             ModelCheckpoint(filepath='best_model.h5', monitor='loss_value', save_best_only=True)]

model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5))

In [ ]:
class_weights = class_weight.compute_class_weight("balanced", classes_to_predict, df_train.label.values)
class_weights_dict = {i : class_weights[i] for i,label in enumerate(classes_to_predict)}

In [ ]:
Teacher_model_hist = model.fit(train_generator,
          epochs = 5, 
          validation_data=validation_generator,
          class_weight=class_weights_dict,
          callbacks=callbacks)